In [ ]:
import pymysql #mysql 연동
import numpy as np #숫자 배열&계산
from scipy.spatial.distance import euclidean #유클리드 거리 계산

#user_record와 album 테이블에서 데이터 가져오는 함수
def fetch_records_and_albums(connection):
    #db연결할 때 커서를 생성하고, sql쿼리를 실행한 후 블록 끝나면 커서가 자동으로 닫힘
    with connection.cursor() as cursor:
        #user_record 테이블에서 user_id와 album_id 가져옴
        cursor.execute("SELECT user_id, album_id FROM user_record")
        records = cursor.fetchall()
        #album 테이블에서 id와 크롤링한 필드 값 가져옴
        cursor.execute("""
            SELECT id, music_acousticness, music_danceability, music_energy, 
                   music_liveness, music_loudness, music_speechiness, 
                   music_tempo, music_valence 
            FROM album
        """)
        albums = cursor.fetchall()
    #결과값 튜플로 반환
    return records, albums

#사용자가 재생한 음악들의 평균 특성 벡터 계산
#album_dict는 앨범id를 키로 하고, 특성 벡터를 값으로 가지는 딕셔너리
def calculate_average_features(user_records, album_dict):
    #첫 번째 앨범의 특성 벡터 길이만큼 0으로 초기화
    feature_sums = np.zeros(len(next(iter(album_dict.values()))))
    #재생한 앨범 수 기록할 변수 0으로 초기화
    count = 0

    #재생 기록된 음악에서 앨범 아이디에 대한 특성 벡터 가져와 배열에 더함
    for user_id, album_id in user_records:
        feature_sums += album_dict[album_id]
        count += 1

    #사용자가 감상한 앨범이 없는 경우, 그대로 반환
    if count == 0:
        return feature_sums
    else:
        #아닌 경우, 평균 특성 벡터 계산
        return feature_sums / count  

#main 함수
def main():
    #mysql 연결
    connection = pymysql.connect(
        host='mitidb.cvm64ss6y2xv.ap-northeast-2.rds.amazonaws.com',       
        user='minseo',  
        password='Alstj!!809', 
        database='mitiDB'
    )

    try:
        #user_record, album 테이블에서 데이터 가져옴
        records, albums = fetch_records_and_albums(connection)

        #album_id를 키로 하고,앨범 특성을 값으로 하는 딕셔너리 생성
        album_dict = {album[0]: np.array(album[1:]) for album in albums}

        #user_record에 있는 모든 user_id에 대해 평균 특성 벡터 계산
        average_features = calculate_average_features(records, album_dict)

        #각 앨범 특성 벡터와 사용자 평균 특성 벡터 간 유클리드 거리 게산 후 리스트에 저장
        similarities = []
        for album_id, features in album_dict.items():
            #유클리드 거리 계산
            euclidean_distance = euclidean(average_features, features)
            #앨범 id, 유클리드 거리 쌍으로 구성하여 저장
            similarities.append((album_id, euclidean_distance))

        #유사도 기준 상위 20개 앨범 선택
        top_20_albums = sorted(similarities, key=lambda x: x[1])[:20]

        #결과 customized_rec 테이블에 삽입
        with connection.cursor() as cursor:
            #모든 user_id에 대해 반복
            for user_id, _ in records:  
                for album_id, distance in top_20_albums:
                    try:
                        cursor.execute(
                            "INSERT INTO customized_rec (user_id, album_id) VALUES (%s, %s)",
                            (user_id, album_id)
                        )
                    #중복 키 에러가 발생시 해당 항목을 삽입하지 않음
                    except pymysql.IntegrityError:
                        continue
            #변경사항을 커밋하여 데이터베이스에 반영
            connection.commit()  

        #결과 출력
        print("Top 20 albums by Euclidean distance have been inserted into customized_rec table.")

    except pymysql.MySQLError as e:
        print(f"Error: {e}")

    finally:
        #데이터베이스 연결 닫기
        connection.close()  

if __name__ == "__main__":
    main()
